In [1]:
import pandas as pd

hadiths = pd.read_csv('kaggle_hadiths_clean.csv')
rawis = pd.read_csv('kaggle_rawis.csv')
# rawi_data = pd.read_csv('rawi_data.csv')

In [13]:
hadiths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34441 entries, 0 to 34440
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          34441 non-null  int64 
 1   hadith_id   34441 non-null  int64 
 2   source      34441 non-null  object
 3   chapter_no  34441 non-null  int64 
 4   hadith_no   34441 non-null  object
 5   chapter     34441 non-null  object
 6   chain_indx  34318 non-null  object
 7   text_ar     34433 non-null  object
 8   text_en     33588 non-null  object
dtypes: int64(3), object(6)
memory usage: 2.4+ MB


In [32]:
hadiths.groupby("source").count().sort_values("hadith_id", ascending=False)

,id,hadith_id,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en
source,,,,,,,,
Sahih Muslim,7596,7596,7596,7596,7596,7558,7590,7242
Sahih Bukhari,7370,7370,7370,7370,7370,7336,7370,7193
Sunan an-Nasa'i,5774,5774,5774,5774,5774,5771,5774,5766
Sunan Abi Da'ud,5260,5260,5260,5260,5260,5253,5260,5260
Sunan Ibn Majah,4227,4227,4227,4227,4227,4211,4227,4099
Jami' al-Tirmidhi,4214,4214,4214,4214,4214,4189,4212,4028


In [28]:
hadiths.head()

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3",حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سف...,Narrated 'Umar bin Al-Khattab: ...
1,1,2,Sahih Bukhari,1,2,Revelation - كتاب بدء الوحى,"30355, 20001, 11065, 10511, 53",حدثنا عبد الله بن يوسف، قال أخبرنا مالك، عن هش...,Narrated 'Aisha: ...
2,2,3,Sahih Bukhari,1,3,Revelation - كتاب بدء الوحى,"30399, 20023, 11207, 11013, 10511, 53",حدثنا يحيى بن بكير، قال حدثنا الليث، عن عقيل، ...,Narrated 'Aisha: (the m...
3,3,4,Sahih Bukhari,1,4,Revelation - كتاب بدء الوحى,"11013, 10567, 34",قال ابن شهاب وأخبرني أبو سلمة بن عبد الرحمن، أ...,Narrated Jabir bin 'Abdullah Al-Ansari while ...
4,4,5,Sahih Bukhari,1,5,Revelation - كتاب بدء الوحى,"20040, 20469, 11399, 11050, 17",حدثنا موسى بن إسماعيل، قال حدثنا أبو عوانة، قا...,Narrated Said bin Jubair: ...


In [18]:
rawis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24326 entries, 0 to 24325
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   scholar_indx          24326 non-null  int64  
 1   name                  24326 non-null  object 
 2   grade                 24326 non-null  object 
 3   parents               3012 non-null   object 
 4   spouse                1010 non-null   object 
 5   siblings              2068 non-null   object 
 6   children              1583 non-null   object 
 7   birth_date_place      7555 non-null   object 
 8   places_of_stay        16728 non-null  object 
 9   death_date_place      24320 non-null  object 
 10  teachers              13436 non-null  object 
 11  students              6499 non-null   object 
 12  area_of_interest      24272 non-null  object 
 13  tags                  15666 non-null  object 
 14  books                 6 non-null      object 
 15  students_inds      

In [34]:
rawis.grade.unique()

array(['Rasool Allah', 'Comp.(RA) [1st Generation]', 'Comp.(RA)',
       'Comp.(RA) [3rd Generation]', 'Comp.(RA) [4th generation]',
       'Comp.(RA) [2nd Generation]', "Follower(Tabi')",
       'Comp.(RA) [6th generation]', 'Comp.(RA) [7th generation]',
       "Prophet's Relative", "Prophet's Relative [Non-Muslim]",
       "Follower(Tabi') [2nd Generation]",
       "Follower(Tabi') [3rd Generation]",
       "Follower(Tabi') [6th generation]",
       "Follower(Tabi') [4th generation]",
       "Follower(Tabi') [5th generation]",
       "Follower(Tabi') [1st Generation]",
       "Follower(Tabi') [8th generation]", "Succ. (Taba' Tabi')",
       "Follower(Tabi') [11th generation]",
       "Follower(Tabi') [7th generation]",
       "Follower(Tabi') [9th generation]",
       "Succ. (Taba' Tabi') [7th generation] [Maliki]",
       "Succ. (Taba' Tabi') [9th generation] [Shafi'ee]",
       "Succ. (Taba' Tabi') [Hanafi]",
       "Succ. (Taba' Tabi') [8th generation]",
       "Succ. (Taba' Tabi'

In [15]:
rawis.columns

Index(['scholar_indx', 'name', 'grade', 'parents', 'spouse', 'siblings',
       'children', 'birth_date_place', 'places_of_stay', 'death_date_place',
       'teachers', 'students', 'area_of_interest', 'tags', 'books',
       'students_inds', 'teachers_inds', 'birth_place', 'birth_date',
       'birth_date_hijri', 'birth_date_gregorian', 'death_date_hijri',
       'death_date_gregorian', 'death_place', 'death_reason'],
      dtype='object')

In [16]:
rawis.scholar_indx.head()

0    1
1    2
2    3
3    4
4    5
Name: scholar_indx, dtype: int64

In [27]:
rawis[rawis['scholar_indx'] == 6]["area_of_interest"]

5    Recitation/Quran, Narrator [ ع  -  صحابة ], Co...
Name: area_of_interest, dtype: object

In [ ]:
rawis[rawis['scholar_indx'] == 3]